In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import orbipy as op
import os

In [15]:
def save_trajectories_data(data, filename="out", path="./data/", plotter=False):
    """
    Сохраняет данные траекторий в CSV файл и, опционально, сохраняет изображение траектории.

    Аргументы:
        data (pd.DataFrame): Данные траектории для сохранения.
        filename (str, optional): Имя файла для сохранения данных. По умолчанию "out".
        path (str, optional): Путь к директории для сохранения файла. По умолчанию "./data/".
        plotter (bool, optional): Если передан объект plotter, сохраняет изображение траектории. По умолчанию False.

    Структура сохранения:
        - Данные сохраняются в формате CSV в указанной директории.
        - Если указан plotter, изображения траекторий сохраняются в поддиректории "images" внутри path.
    """
    
    if not os.path.exists(path):
        os.mkdir(path)
    if not os.path.exists(f"{path}images/"):
        os.mkdir(f"{path}images/")

    if plotter:
        axis = plotter.plot_proj(data)
        plotter.plot_proj(plottables=[plotter.m, plotter.L1], marker='.', ax=axis)
        plt.savefig(f"{path}images/{filename}.png", format="png")
        plt.close()
    pd.DataFrame.to_csv(data, f"{path}{filename}.csv")

def generate_and_save_trajectory_data(model, start_data, filename, path, is_plotting):
    """
    Генерирует данные траектории на основе начальных условий и сохраняет их в файл.

    Аргументы:
        start_data (array-like): Начальные условия для генерации траектории.
        is_plotting (bool): Указывает, нужно ли сохранять изображение траектории.
    
    Генерирует данные траектории на основе начальных условий, и сохраняет данные 
        в CSV файл и, при необходимости, изображение траектории.
    """
    if is_plotting:
        plotter = op.plotter.from_model(model, length_units='Mm')
    else:
        plotter = None
        
    start_state = model.get_zero_state()
    start_state[[0, 2, 4]] = start_data[1:]

    #TODO: Указать кол-во пересечений (цифра 2) в аргументах (в идеале как-то узнавать процедурно?)
    detector = op.event_detector(model, events=[op.eventY(count=2)])
    data, _ = detector.prop(start_state, 0, 5*np.pi, last_state='last')

    save_trajectories_data(data, filename, path, plotter=plotter)

def save_trajectories(model, start_data, path, csv_name, is_plotting):
    """
    Сохраняет данные траекторий для набора начальных условий.

    Аргументы:
        model (object): Модель, используемая для генерации данных траектории.
        start_data (pd.DataFrame): DataFrame с начальными условиями для генерации траекторий.
        path (str): Основной путь для сохранения данных.
        csv_name (str): Название поддиректории для сохранения CSV файлов.
        is_plotting (bool): Указывает, нужно ли сохранять изображения траекторий.

    Стуктура файлов выглядит так:
        - `/path/{csv_name}/{i}_[{start_data}].csv`
        - `/path/{csv_name}/images/{i}_[{start_data}].csv`
    """
    for data in start_data[['x', 'z', 'v']].head(10).itertuples(index=True, name=None): #.head(10)
        # print(data)
        filename = f"{data[0]}_[{data[1]}_{data[2]}_{data[3]}]"
        generate_and_save_trajectory_data(model, data, filename, f"{path}{csv_name}/", is_plotting)

In [10]:
def generate_poincare_sections_data(model, start_data, events):
    """
    Генерирует данные сечений Пуанкаре для начального условия и плоскостей сечения.

    Аргументы:
        model (object): Модель, используемая для генерации данных.
        start_data (pd.DataFrame): DataFrame с начальными условиями для генерации траекторий.
        events ([op.eventX, op.eventY, ...]): Массив с событиями - пересечение орбитой указанной плоскости.
    """
    start_state = model.get_zero_state()
    start_state[[0, 2, 4]] = start_data[1:]
    
    detector = op.event_detector(model, events=events)
    # print(start_state)
    data, event_points = detector.prop(start_state, 0, 5 * np.pi, last_state='last')
    return event_points

def generate_and_save_poincare_sections_data(model, start_data, path):
    """
    Генерирует данные сечений Пуанкаре для начального условия и плоскостей и сохраняет их в файлы.

    Аргументы: 
        model (object): Модель, используемая для генерации данных.
        start_data (pd.DataFrame): DataFrame с начальными условиями для генерации сечений.
        path (String): Путь для сохранения данных.
    
    Генерирует данные сечений Пуанкаре и сохраняет их в отдельные файлы с соответствующими названиями.
    """
    if not os.path.exists(path):
        os.mkdir(path)
    
    events = [op.eventX(count=2, terminal=False), op.eventY(count=2, terminal=False), op.eventZ(count=2, terminal=False), op.eventVX(count=2, terminal=False), op.eventVY(count=2, terminal=False), op.eventVZ(count=2, terminal=False)]
    event_points = generate_poincare_sections_data(model, start_data, events)
    # print(event_points)
    for key, value in {0.0: "X", 1.0: "Y", 2.0: "Z", 3.0: "VX", 4.0: "VY", 5.0: "VZ"}.items():
        if not event_points.loc[event_points['e'] == key].empty:
            pd.DataFrame.to_csv(event_points.loc[event_points['e'] == key],
                                f"{path}{value}_[{start_data[1]}_{start_data[2]}_{start_data[3]}].csv")
    
def save_poincare_sections(model, start_data, initial_path, csv_name):
    """
    Генерирует данные сечений Пуанкаре для набора начальных условий и плоскостей и сохраняет их в файлы.

    Аргументы: 
        model (object): Модель, используемая для генерации данных.
        start_data (pd.DataFrame): DataFrame с начальными условиями для генерации сечений.
        initial_path (String): Основной путь для сохранения данных.
        csv_name (String): Название csv-файла с данными.

    Стуктура файлов выглядит так:
        - `/initial_path/{csv_name}/poincare/{i}/{crosssection_plane}_[{start_data}].csv`
    """
    path = f"{initial_path}{csv_name}/poincare/"
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

    for data in start_data[['x', 'z', 'v']].head(10).itertuples(index=True, name=None):
        # print(data)
        generate_and_save_poincare_sections_data(model, data, f"{path}{data[0]}/")


In [11]:
model = op.crtbp3_model('Earth-Moon (default)')
halo = pd.read_csv("halo_general_low_l1.csv")

In [18]:
save_trajectories(model, halo, "./data/", "halo_general_low_l1", False)

In [17]:
save_poincare_sections(model, halo, "./data/", "halo_general_low_l1")